# Replikation von Tabelle 13 und 14 aus dem Paper (TinyStories)

## Ziel
Berechnung von:
- **Top-1 Accuracy** (Tabelle 13)
- **Variational Distance** (Tabelle 14)  
für ein eigenes Modell auf dem **TinyStories-Datensatz**.

## Vorgehen

### 1. Laden der Regel-Daten
- Die Regel-Daten enthalten Kontexte und N-Gram-Zähler (`next_token_counter`), die aus TinyStories extrahiert wurden.
- Sie dienen als Referenz zur Bewertung des Modells.

### 2. Modellvorhersagen berechnen
- Für jeden Kontext wird das Modell verwendet, um die **Wahrscheinlichkeitsverteilung für das nächste Token** zu berechnen.
- Daraus wird das **Top-1 Token des Modells** bestimmt.

### 3. Vergleich mit Regeln
- Die **Regel-Zähler** werden in Wahrscheinlichkeiten umgerechnet.
- Dann wird geprüft, ob das Modell **das gleiche Top-1 Token** wie die Regel vorhersagt.
- Zusätzlich wird die **variationale Distanz** zwischen Modell und Regel berechnet.

### 4. Gruppierung nach Kontextlänge
- Die Ergebnisse werden nach der **Kontextlänge (1–7 Tokens)** gruppiert.
- Für jede Länge werden die **durchschnittliche Top-1 Accuracy** und **Distanz** berechnet.

### 5. Export im Paper-Stil
- Die Ergebnisse werden in zwei Tabellen im Format des Papers gespeichert:
    - **Zeilen:** Modellgröße (hier nur „124M“)
    - **Spalten:** Kontextlängen **1–7**
.

### 6. Speichern als CSV
- Die Tabellen werden als **CSV-Dateien** gespeichert:
    - `table_top1_accuracy.csv`
    - `table_distance.csv`

## Beispiel: Tabellenstruktur

| Model Size / Context Length | 1   | 2   | 3   | 4   | 5   | 6   | 7   |
|-----------------------------|-----|-----|-----|-----|-----|-----|-----|
| 124M                        | x.x | x.x | x.x | x.x | x.x | x.x | x.x |



In [2]:
import torch
import pandas as pd
import numpy as np
import sentencepiece as spm
import gcsfs
from transformers import AutoModelForCausalLM

# GPU aktivieren, falls verfügbar
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Google Cloud Storage verbinden
fs = gcsfs.GCSFileSystem('transformer-ngrams')

# Tokenizer laden
TOKENIZER_PATH = 'gs://transformer-ngrams/32768.model'
VOCAB_SIZE = 32768
BOS_TOKEN = 1
with fs.open(TOKENIZER_PATH) as f:
    tokenizer = spm.SentencePieceProcessor(model_proto=f.read())

# Transformer-Modell laden, unser trainiertes Modell
model_name = "dadosbon/TSModel2_124M"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Funktion für Modellvorhersagen (angepasst für Geschwindigkeit)
def get_model_predictions(input_tokens):
    '''
    Gibt die Wahrscheinlichkeitsverteilung für das nächste Token zurück,
    basierend auf dem gegebenen Kontext (input_tokens).

    Parameter:
        input_tokens (list of int): Liste von Token-IDs als Kontext.

    Rückgabe:
        numpy.ndarray: Array der Wahrscheinlichkeiten (Softmax) über das Vokabular.
    '''
    input_tensor = torch.tensor([input_tokens]).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)
        logits = outputs.logits[:, -1, :VOCAB_SIZE]
        probs = torch.nn.functional.softmax(logits, dim=-1)

                # Debugging-Ausgabe
        print("Logits (min, max):", logits.min().item(), logits.max().item())
        print("Erste 3 Logits:", logits[0, :3])
    return probs.cpu().numpy().flatten()

def convert_counter_to_probs(arr):
    '''
    Wandelt ein N-Gram Counter-Array in eine Wahrscheinlichkeitsverteilung um.

    Parameter:
        arr (list or numpy.ndarray): Abwechselnd Token-ID und Count, z.B. [id1, count1, id2, count2, ...].
        k (int): Anzahl der Top-k Tokens, die für Debugging extrahiert werden (optional, hier nicht verwendet).

    Rückgabe:
        numpy.ndarray: Wahrscheinlichkeiten über das Vokabular, basierend auf N-Gram Counts.
    '''
    probs = np.zeros(VOCAB_SIZE)

    if arr is None or len(arr) == 0:
        return probs  # Falls leer, nur Nullen zurückgeben!

    arr = np.array(arr).flatten()

    # Falls die Länge ungerade ist, entfernen wir das letzte Element
    if len(arr) % 2 != 0:
        arr = arr[:-1]

    try:
        tokens = arr[::2].astype(int)  # Token-IDs
        counts = arr[1::2].astype(int)  # Häufigkeiten
    except Exception as e:
        print("Fehler beim Extrahieren von Tokens/Counts:", e)
        return probs  # Falls ein Fehler auftritt, bleibt es 0


    if len(tokens) == 0 or len(counts) == 0:
        return probs  # Falls etwas schiefgeht, alle Wahrscheinlichkeiten auf 0 setzen

    total_count = np.sum(counts)
    if total_count == 0:
        return probs  # Falls alle Counts 0 sind, bleibt es 0!

    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts) if count > 0}

    for token, prob in token_probs.items():
        if 0 <= token < VOCAB_SIZE:
            probs[token] = prob  # Wahrscheinlichkeiten setzen

    return probs  # Falls leer, nur Nullen zurückgeben!

    arr = np.array(arr).flatten()

    # Falls die Länge ungerade ist, entfernen wir das letzte Element
    if len(arr) % 2 != 0:
        arr = arr[:-1]

    tokens = arr[::2]  # Token-IDs
    counts = arr[1::2]  # Häufigkeiten

    if len(tokens) == 0 or len(counts) == 0:
        return probs  # Falls etwas schiefgeht, alle Wahrscheinlichkeiten auf 0 setzen

    total_count = np.sum(counts)
    if total_count == 0:
        return probs  # Falls alle Counts 0 sind, bleibt es 0!

    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts) if count > 0}
    topk_tokens = sorted(token_probs, key=token_probs.get, reverse=True)[:k]

    for token in topk_tokens:
        if 0 <= token < VOCAB_SIZE:
            probs[token] = token_probs[token]  # Wahrscheinlichkeiten setzen

    return probs  # Falls leer, nur Nullen zurückgeben!

    arr = np.array(arr).flatten()

    # Falls die Länge ungerade ist, entfernen wir das letzte Element
    if len(arr) % 2 != 0:
        arr = arr[:-1]

    tokens = arr[::2]  # Token-IDs
    counts = arr[1::2]  # Häufigkeiten

    total_count = np.sum(counts)
    if total_count == 0:
        return probs  # Falls alle Counts 0 sind, bleibt es 0!

    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts)}
    topk_tokens = sorted(token_probs, key=token_probs.get, reverse=True)[:k]

    for token in topk_tokens:
        if 0 <= token < VOCAB_SIZE:
            probs[token] = token_probs[token]  # Wahrscheinlichkeiten setzen

    return probs
    arr = np.array(arr).flatten()
    if arr.ndim != 1 or len(arr) % 2 != 0:
        return probs
    tokens = arr[::2]
    counts = arr[1::2]
    total_count = np.sum(counts)
    if total_count == 0:
        return probs
    token_probs = {int(token): count / total_count for token, count in zip(tokens, counts)}
    topk_tokens = sorted(token_probs, key=token_probs.get, reverse=True)[:k]
    for token in topk_tokens:
        if token < VOCAB_SIZE:
            probs[token] = token_probs[token]
    return probs

def dist(counter, model_probs):
    '''
    Berechnet die Variationale Distanz zwischen Regel-Prediktion und Modellvorhersage.

    Parameter:
        counter (list or numpy.ndarray): N-Gram Counter für die Regel (siehe oben).
        model_probs (numpy.ndarray): Wahrscheinlichkeitsverteilung vom Modell.

    Rückgabe:
        float: Variationale Distanz (0 bis 1) zwischen Regel und Modell.
    '''
    probs = convert_counter_to_probs(counter)
    return 0.5 * np.sum(np.abs(probs - model_probs[:VOCAB_SIZE]))

# Regeln-Daten laden AUSSCHNITT
print("Laden der Regeln-Daten für Tabelle 13, 14...")
sample_rules_path = 'gs://transformer-ngrams/TinyStories/eval_data_rules/001.parquet'
with fs.open(sample_rules_path, 'rb') as f:
    df_rules = pd.read_parquet(f)
print("Regeln-Daten geladen:", df_rules.shape)

# Modellvorhersagen AUSSCHNITT
df_rules_sample = df_rules.sample(n=1000, random_state=42).copy()
model_probs = []
for token_list in df_rules_sample["token"]:
    probs = get_model_predictions([token_list])
    model_probs.append(probs.tolist())


'''#FÜR ALLE DATEN DANN:                    #Die beiden vorhergehenden Blöcke mit dem ersetzen, für ALLE daten
# Alle Regeln-Daten laden
print("Lade alle Regeln-Daten für Tabelle 13 & 14...")
parquet_files = fs.ls('transformer-ngrams/TinyStories/eval_data_rules/')
df_list = []

for path in parquet_files:
    with fs.open(f'gs://{path}', 'rb') as f:
        df_list.append(pd.read_parquet(f))

df_rules = pd.concat(df_list, ignore_index=True)
print("Alle Regeln-Daten geladen:", df_rules.shape)
df_rules_sample = df_rules.copy()
model_probs = []
for token_list in df_rules_sample["token"]:
    probs = get_model_predictions([token_list])
    model_probs.append(probs.tolist())'''



df_rules_sample["model_probs"] = model_probs

df_rules_sample["distance"] = df_rules_sample.apply(
    lambda row: dist(row["next_token_counter"], row["model_probs"]), axis=1
)

df_rules_sample["rule_prediction"] = df_rules_sample["next_token_counter"].apply(convert_counter_to_probs)
df_rules_sample["model_top_1"] = df_rules_sample["model_probs"].apply(np.argmax)
df_rules_sample["matches_rule"] = df_rules_sample.apply(
    lambda x: x["model_top_1"] in np.argsort(x["rule_prediction"])[-1:], axis=1
)

# Gruppierung nach Kontextlänge für Tabellen 13 & 14
df_rules_sample["context_length"] = df_rules_sample["context"].apply(
    lambda x: len([t for t in x.tolist() if t != 0]) if isinstance(x, np.ndarray) else 1
)
'''table_results = df_rules_sample.groupby("context_length").agg(
    top1_accuracy=("matches_rule", "mean"),
    avg_distance=("distance", "mean")
).reset_index()

table_results.rename(columns={"context_length": "Regeln / Kontextlänge", 
                               "top1_accuracy": "Top-1 Genauigkeit", 
                               "avg_distance": "Variationale Distanz"}, inplace=True)'''



# Modellname festlegen
model_name = "124M"


agg = df_rules_sample.groupby("context_length").agg(
    top1_accuracy=("matches_rule", "mean"),
    avg_distance=("distance", "mean")
).reset_index()

# Tabellen vorbereiten
# → Zeile: Modellname | Spalten: Kontextlängen 1–7

# Top-1 Accuracy Tabelle
accuracy_row = agg.set_index("context_length")["top1_accuracy"]
accuracy_table = pd.DataFrame([accuracy_row], index=[model_name])
accuracy_table.columns.name = "Context Length"

# Distanz Tabelle
distance_row = agg.set_index("context_length")["avg_distance"]
distance_table = pd.DataFrame([distance_row], index=[model_name])
distance_table.columns.name = "Context Length"

# Zeilenindex benennen – das ist die linke obere Zelle!
accuracy_table.index.name = "Model Size / Context Length"
distance_table.index.name = "Model Size / Context Length"




# Ausgabe prüfen
print("\n📊 Top-1 Accuracy Tabelle:")
print(accuracy_table.round(3))

print("\n📊 Distanz Tabelle:")
print(distance_table.round(3))

# Optional: Als CSV speichern
accuracy_table.to_csv("table_top1_accuracy.csv")
distance_table.to_csv("table_distance.csv")



# Speichern der Ergebnisse
#table_results.to_csv("tiny_stories_results.csv", index=False)


Using device: cuda
Laden der Regeln-Daten für Tabelle 13, 14...
Regeln-Daten geladen: (28073, 11)
Logits (min, max): -42.66048812866211 -18.35240936279297
Erste 3 Logits: tensor([-27.5445, -21.9545, -38.0549], device='cuda:0')
Logits (min, max): -40.68497848510742 -18.37091827392578
Erste 3 Logits: tensor([-27.8901, -22.7580, -36.7036], device='cuda:0')
Logits (min, max): -49.81781005859375 -23.027111053466797
Erste 3 Logits: tensor([-33.0452, -28.6488, -43.4766], device='cuda:0')
Logits (min, max): -49.277305603027344 -22.447681427001953
Erste 3 Logits: tensor([-32.2455, -27.6965, -43.3676], device='cuda:0')
Logits (min, max): -40.68497848510742 -18.37091827392578
Erste 3 Logits: tensor([-27.8901, -22.7580, -36.7036], device='cuda:0')
Logits (min, max): -42.17828369140625 -17.289297103881836
Erste 3 Logits: tensor([-26.3385, -21.9841, -36.9996], device='cuda:0')
Logits (min, max): -28.308948516845703 -5.624163627624512
Erste 3 Logits: tensor([-13.2697,  -8.8970, -23.7738], device='cud

In [3]:
# Zeige 5 Beispiele mit Modell vs. Regel | wie gut performt das Modell?
for idx, row in df_rules_sample.head(5).iterrows():
    context_tokens = row["token"]
    model_probs = row["model_probs"]
    rule_probs = row["rule_prediction"]

    model_top1 = np.argmax(model_probs)
    rule_top1 = np.argmax(rule_probs)

    print(f"\n Beispiel {idx}")
    print("Kontext (Token-IDs):", context_tokens)
    print("Kontext (Text):", tokenizer.decode(context_tokens))
    print("Model Top-1 ID:", model_top1, "| Token:", tokenizer.id_to_piece(int(model_top1)))
    print("Rule Top-1 ID:", rule_top1, "| Token:", tokenizer.id_to_piece(int(rule_top1)))

    print("Match?:", model_top1 == rule_top1)

    # Optional: Wahrscheinlichkeiten
    print("Model Top-1 Wahrscheinlichkeit:", model_probs[model_top1])
    print("Rule Top-1 Wahrscheinlichkeit:", rule_probs[rule_top1])



 Beispiel 10034
Kontext (Token-IDs): 32600
Kontext (Text): ,
Model Top-1 ID: 280 | Token: ▁the
Rule Top-1 ID: 32604 | Token: I
Match?: False
Model Top-1 Wahrscheinlichkeit: 0.13138867914676666
Rule Top-1 Wahrscheinlichkeit: 0.305989110707804

 Beispiel 22703
Kontext (Token-IDs): 305
Kontext (Text):  and
Model Top-1 ID: 280 | Token: ▁the
Rule Top-1 ID: 305 | Token: ▁and
Match?: False
Model Top-1 Wahrscheinlichkeit: 0.14320680499076843
Rule Top-1 Wahrscheinlichkeit: 0.14917421417155036

 Beispiel 14069
Kontext (Token-IDs): 3373
Kontext (Text):  playing
Model Top-1 ID: 32599 | Token: .
Rule Top-1 ID: 1947 | Token: One
Match?: False
Model Top-1 Wahrscheinlichkeit: 0.28632161021232605
Rule Top-1 Wahrscheinlichkeit: 0.5960715185091916

 Beispiel 11812
Kontext (Token-IDs): 364
Kontext (Text):  it
Model Top-1 ID: 32599 | Token: .
Rule Top-1 ID: 459 | Token: ▁can
Match?: False
Model Top-1 Wahrscheinlichkeit: 0.3384929895401001
Rule Top-1 Wahrscheinlichkeit: 0.5

 Beispiel 667
Kontext (Token-ID